In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from keras.models import Sequential, Model
from keras.layers import Dense, LSTM, Conv2D, Dropout, MaxPool2D, Flatten
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor

Using TensorFlow backend.


In [2]:
train = pd.read_csv('train.csv', index_col=0)
test = pd.read_csv('test.csv', index_col=0)
submission = pd.read_csv('sample_submission.csv', index_col=0)

In [8]:
train.iloc[:,:30]

,rho,650_src,660_src,670_src,680_src,690_src,700_src,710_src,720_src,730_src,...,840_src,850_src,860_src,870_src,880_src,890_src,900_src,910_src,920_src,930_src
id,,,,,,,,,,,,,,,,,,,,,
0,25,0.37950,0.42993,0.52076,0.57166,0.67818,0.75476,0.83580,0.93623,0.96333,...,0.34935,0.30038,0.26992,0.23192,0.21998,0.19920,0.14493,0.13150,0.12442,0.10786
1,10,0.00000,0.00000,0.01813,0.00000,0.00000,0.01974,0.00321,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.04466,0.10738,0.21646,0.41456,0.63716,0.84230,0.97480
2,25,0.00000,0.03289,0.02416,0.03610,0.05843,0.09015,0.14944,0.18578,0.25584,...,0.98988,0.95101,0.89853,0.79813,0.68262,0.58043,0.46222,0.39113,0.27879,0.19070
3,10,0.27503,0.31281,0.32898,0.41041,0.46587,0.52769,0.64369,0.73562,0.79865,...,0.43087,0.35684,0.30786,0.27722,0.24119,0.20158,0.17876,0.15139,0.15656,0.13392
4,15,1.01521,1.00872,0.98930,0.98874,1.01773,1.01632,1.00009,0.98217,1.01564,...,0.98861,0.98270,1.00432,1.01434,1.01455,0.98861,0.98300,1.00616,0.99045,1.01321
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,15,0.23929,0.30265,0.39929,0.51000,0.64072,0.77328,0.86722,0.95891,0.98998,...,0.11772,0.08974,0.03196,0.04683,0.00020,0.00389,0.00000,0.00317,0.00811,0.00000
9996,20,0.02583,0.00946,0.03650,0.01380,0.04093,0.04363,0.03260,0.05282,0.04608,...,0.29630,0.41237,0.55772,0.73862,0.94945,0.99104,0.89597,0.69515,0.53156,0.40282
9997,10,0.57589,0.62976,0.70571,0.77248,0.85402,0.92796,0.97691,0.98933,1.00493,...,0.38292,0.32223,0.28090,0.26486,0.25972,0.21750,0.19588,0.17659,0.17717,0.14670


In [ ]:
# 맷플로립으로 확인해야 할 듯

In [38]:
train = train.interpolate()
test = test.interpolate()

train = train.fillna(train.mean())
test = test.fillna(train.mean())

print(train.shape)
print(test.shape)

train = np.array(train)
x_predict = np.array(test)

x = train[:,:71]
y = train[:,71:]

# 전처리
from sklearn.preprocessing import RobustScaler, StandardScaler
scaler = RobustScaler()
scaler.fit(x) 
x = scaler.transform(x)
x_predict = scaler.transform(x_predict)

# from sklearn.decomposition import PCA
# pca = PCA(n_components=12)
# pca.fit(x)
# x = pca.transform(x)
# x_predict = pca.transform(x_predict)

(10000, 75)
(10000, 71)


In [167]:
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# scaler.fit(x) 
# x = scaler.transform(x)
# x_predict = scaler.transform(x_predict)

In [2]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=43)
model = RandomForestRegressor(n_estimators=160)
model.fit(x_train,y_train)
y_pred = model.predict(x_test) # 회귀던 분류던 사용할 수 있음
mae = mean_absolute_error(y_pred, y_test)
print('랜덤포레스트 mae 는', mae)

랜덤포레스트 mae 는 1.4601283828124996


# GradientBoostingRegressor

In [9]:
y[:,0]

array([ 5.59,  0.  , 10.64, ...,  9.84,  6.38,  9.35])

In [10]:
a = []
for i in range(y.shape[1]) :
    a.append(y[:,i])

In [11]:
a

[array([ 5.59,  0.  , 10.64, ...,  9.84,  6.38,  9.35]),
 array([4.32, 2.83, 3.  , ..., 3.2 , 4.06, 4.34]),
 array([ 8.92,  7.25,  8.4 , ..., 10.45, 11.28,  9.73]),
 array([4.29, 4.64, 5.16, ..., 2.06, 4.03, 3.54])]

In [29]:
x_train, x_test, y_train, y_test = train_test_split(x,a[0], test_size=0.2, random_state=43)

model = GradientBoostingRegressor()
model.fit(x_train,y_train)
y_pred = model.predict(x_test) # 회귀던 분류던 사용할 수 있음
mae = mean_absolute_error(y_pred, y_test)
print('GradientBoostingRegressor mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,a[1], test_size=0.2, random_state=43)
model = GradientBoostingRegressor()
model.fit(x_train,y_train)
y_pred = model.predict(x_test) # 회귀던 분류던 사용할 수 있음
mae = mean_absolute_error(y_pred, y_test)
print('GradientBoostingRegressor mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,a[2], test_size=0.2, random_state=43)
model = GradientBoostingRegressor()
model.fit(x_train,y_train)
y_pred = model.predict(x_test) # 회귀던 분류던 사용할 수 있음
mae = mean_absolute_error(y_pred, y_test)
print('GradientBoostingRegressor mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,a[3], test_size=0.2, random_state=43)
model = GradientBoostingRegressor()
model.fit(x_train,y_train)
y_pred = model.predict(x_test) # 회귀던 분류던 사용할 수 있음
mae = mean_absolute_error(y_pred, y_test)
print('GradientBoostingRegressor mae 는', mae)

GradientBoostingRegressor mae 는 1.5838840045681222
GradientBoostingRegressor mae 는 0.7350616301783675
GradientBoostingRegressor mae 는 2.308885373741608
GradientBoostingRegressor mae 는 1.4267326043872253


In [13]:
x_train, x_test, y_train, y_test = train_test_split(x,a[0], test_size=0.2, random_state=43)

model = GradientBoostingRegressor(n_estimators=200)
model.fit(x_train,y_train)
y_pred = model.predict(x_test) # 회귀던 분류던 사용할 수 있음
mae = mean_absolute_error(y_pred, y_test)
print('GradientBoostingRegressor mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,a[1], test_size=0.2, random_state=43)
model = GradientBoostingRegressor(n_estimators=200)
model.fit(x_train,y_train)
y_pred = model.predict(x_test) # 회귀던 분류던 사용할 수 있음
mae = mean_absolute_error(y_pred, y_test)
print('GradientBoostingRegressor mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,a[2], test_size=0.2, random_state=43)
model = GradientBoostingRegressor(n_estimators=200)
model.fit(x_train,y_train)
y_pred = model.predict(x_test) # 회귀던 분류던 사용할 수 있음
mae = mean_absolute_error(y_pred, y_test)
print('GradientBoostingRegressor mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,a[3], test_size=0.2, random_state=43)
model = GradientBoostingRegressor(n_estimators=200)
model.fit(x_train,y_train)
y_pred = model.predict(x_test) # 회귀던 분류던 사용할 수 있음
mae = mean_absolute_error(y_pred, y_test)
print('GradientBoostingRegressor mae 는', mae)

In [20]:
from sklearn.decomposition import PCA
pca = PCA(n_components=1)
pca.fit(y)
y1 = pca.transform(y)
x_train, x_test, y_train, y_test = train_test_split(x,y1, test_size=0.2, random_state=43)
model = GradientBoostingRegressor(n_estimators=200)
model.fit(x_train,y_train)
y_pred = model.predict(x_test) # 회귀던 분류던 사용할 수 있음
mae = mean_absolute_error(y_pred, y_test)
print('GradientBoostingRegressor mae 는', mae)

C:\Users\bitcamp\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


랜덤포레스트 mae 는 1.837030398551365


# XGBRegressor()

In [27]:
x_train, x_test, y_train, y_test = train_test_split(x,a[0], test_size=0.2, random_state=43)

model = XGBRegressor()
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
mae = mean_absolute_error(y_pred, y_test)
print('xgboost mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,a[1], test_size=0.2, random_state=43)
model = XGBRegressor()
model.fit(x_train,y_train)
y_pred = model.predict(x_test) 
mae = mean_absolute_error(y_pred, y_test)
print('xgboost mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,a[2], test_size=0.2, random_state=43)
model = XGBRegressor()
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
mae = mean_absolute_error(y_pred, y_test)
print('xgboost mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,a[3], test_size=0.2, random_state=43)
model = XGBRegressor()
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
mae = mean_absolute_error(y_pred, y_test)
print('xgboost mae 는', mae)

xgboost mae 는 1.2829265556406977
xgboost mae 는 0.7349076338338851
xgboost mae 는 2.2790482931423184
xgboost mae 는 1.4711147229343653


In [32]:
x_train, x_test, y_train, y_test = train_test_split(x,a[0], test_size=0.2, random_state=43)

model = XGBRegressor(n_estimators=200)
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
mae = mean_absolute_error(y_pred, y_test)
print('xgboost mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,a[1], test_size=0.2, random_state=43)
model = XGBRegressor(n_estimators=200)
model.fit(x_train,y_train)
y_pred = model.predict(x_test) 
mae = mean_absolute_error(y_pred, y_test)
print('xgboost mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,a[2], test_size=0.2, random_state=43)
model = XGBRegressor(n_estimators=200)
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
mae = mean_absolute_error(y_pred, y_test)
print('xgboost mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,a[3], test_size=0.2, random_state=43)
model = XGBRegressor(n_estimators=200)
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
mae = mean_absolute_error(y_pred, y_test)
print('xgboost mae 는', mae)

xgboost mae 는 1.283430913808942
xgboost mae 는 0.7389190593361854
xgboost mae 는 2.2966433312320707
xgboost mae 는 1.4670573375791311


In [30]:
from sklearn.decomposition import PCA
pca = PCA(n_components=1)
pca.fit(y)
y1 = pca.transform(y)
x_train, x_test, y_train, y_test = train_test_split(x,y1, test_size=0.2, random_state=43)
model = XGBRegressor()
model.fit(x_train,y_train)
y_pred = model.predict(x_test) # 회귀던 분류던 사용할 수 있음
mae = mean_absolute_error(y_pred, y_test)
print('xgboost mae 는', mae)

xgboost mae 는 1.8154817496832283


# RandomForestRegressor

In [31]:
x_train, x_test, y_train, y_test = train_test_split(x,a[0], test_size=0.2, random_state=43)

model = RandomForestRegressor(n_estimators=200)
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
mae = mean_absolute_error(y_pred, y_test)
print('RandomForestRegressor mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,a[1], test_size=0.2, random_state=43)
model = RandomForestRegressor(n_estimators=200)
model.fit(x_train,y_train)
y_pred = model.predict(x_test) 
mae = mean_absolute_error(y_pred, y_test)
print('RandomForestRegressor mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,a[2], test_size=0.2, random_state=43)
model = RandomForestRegressor(n_estimators=200)
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
mae = mean_absolute_error(y_pred, y_test)
print('RandomForestRegressor mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,a[3], test_size=0.2, random_state=43)
model = RandomForestRegressor(n_estimators=200)
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
mae = mean_absolute_error(y_pred, y_test)
print('RandomForestRegressor mae 는', mae)

xgboost mae 는 1.289325675
xgboost mae 는 0.7111221249999999
xgboost mae 는 2.187418975
xgboost mae 는 1.407558775


In [33]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=43)
model = RandomForestRegressor(n_estimators=160)
model.fit(x_train,y_train)
y_pred = model.predict(x_test) 
mae = mean_absolute_error(y_pred, y_test)
print('랜덤포레스트 mae 는', mae)

랜덤포레스트 mae 는 1.4622248984374993


In [37]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=43)
model = RandomForestRegressor(n_estimators=200)
model.fit(x_train,y_train)
y_pred = model.predict(x_test) 
mae = mean_absolute_error(y_pred, y_test)
print('랜덤포레스트 mae 는', mae)

랜덤포레스트 mae 는 1.7716422312499989


In [39]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=20)
model1 = RandomForestRegressor(n_estimators=250)
model1.fit(x_train,y_train)
y_pred = model1.predict(x_test) 
mae = mean_absolute_error(y_pred, y_test)
print('랜덤포레스트 mae 는', mae)

랜덤포레스트 mae 는 1.441592375


In [ ]:
pred = model1.predict(x_predict)
submission = pd.read_csv('sample_submission.csv', index_col=0)
submission['hhb'] = pred[:,0]
submission['hbo2'] = pred[:,1]
submission['ca'] = pred[:,2]
submission['na'] = pred[:,3]

('sample_submission.csv', index_col=0)

# 제출

In [42]:
# 제출
pred = model1.predict(x_predict)

pred = model1.predict(x_predict)
submission = pd.read_csv('sample_submission.csv', index_col=0)
submission['hhb'] = pred[:,0]
submission['hbo2'] = pred[:,1]
submission['ca'] = pred[:,2]
submission['na'] = pred[:,3]

submission.to_csv('submission0611_1.csv')

In [255]:
submission

,hhb,hbo2,ca,na
id,,,,
10000,7.330,3.861,7.608,2.890
10001,6.530,3.993,6.310,2.105
10002,11.439,4.057,9.335,3.524
10003,8.239,4.599,9.715,3.084
10004,8.279,3.615,9.837,4.130
...,...,...,...,...
19995,6.134,4.480,9.480,3.342
19996,7.258,4.027,9.550,3.275
19997,9.724,3.333,9.786,3.206
